# Day 17: Gemini 3 Flash - 可配置思考级别

Gemini 3 Flash 最大亮点：**Thinking Level（思考级别）**

简单说：控制模型"想多久"再回答。

## 1. 思考级别一览

| 级别 | 特点 | 适用场景 |
|------|------|----------|
| `MINIMAL` | 最快响应 | 简单问答、聊天 |
| `LOW` | 快速 + 少量推理 | 日常任务 |
| `MEDIUM` | 平衡模式 | 中等复杂度 |
| `HIGH` | 深度推理 | 数学、编程、复杂问题 |

## 2. 环境准备

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

# 确认 API Key 已设置
assert os.getenv("GOOGLE_API_KEY"), "请设置 GOOGLE_API_KEY"

In [2]:
from google.adk.agents import Agent
from google.adk.planners import BuiltInPlanner
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
from google.genai.types import Content, Part

## 3. 核心配置：ThinkingConfig

关键代码就这几行：

In [3]:
# 配置思考级别
thinking_config = types.ThinkingConfig(
    thinking_level="LOW"  # 可选: MINIMAL, LOW, MEDIUM, HIGH
)

# 创建规划器
planner = BuiltInPlanner(thinking_config=thinking_config)

print(f"思考级别: {thinking_config.thinking_level}")

思考级别: ThinkingLevel.LOW


## 4. 定义工具

In [4]:
def get_current_time(city: str) -> dict:
    """获取指定城市的当前时间"""
    times = {
        "北京": "22:30",
        "东京": "23:30", 
        "纽约": "09:30",
        "伦敦": "14:30",
    }
    time = times.get(city, "10:00")
    return {"city": city, "time": time}


def calculate(expression: str) -> dict:
    """计算数学表达式"""
    try:
        result = eval(expression)
        return {"expression": expression, "result": result}
    except Exception as e:
        return {"error": str(e)}

## 5. 创建 Agent 工厂函数

In [5]:
def create_agent(thinking_level: str) -> Agent:
    """创建指定思考级别的 Agent"""
    return Agent(
        model='gemini-3-flash-preview',
        name=f'agent_{thinking_level.lower()}',
        instruction="你是一个有用的助手。请用中文简洁回答。",
        tools=[get_current_time, calculate],
        planner=BuiltInPlanner(
            thinking_config=types.ThinkingConfig(
                thinking_level=thinking_level
            )
        ),
    )

print("✅ 工厂函数已定义")

✅ 工厂函数已定义


## 6. 运行辅助函数（含思考过程显示）

关键：通过 `event.content.parts` 中的 `thought` 属性获取思考内容

In [6]:
session_service = InMemorySessionService()

async def ask(agent: Agent, question: str, show_thinking: bool = True) -> str:
    """
    向 Agent 提问并返回回答
    
    参数:
        show_thinking: 是否显示思考过程
    """
    session = await session_service.create_session(
        app_name="day17_demo",
        user_id="demo_user"
    )
    
    runner = Runner(
        agent=agent,
        app_name="day17_demo", 
        session_service=session_service
    )
    
    user_content = Content(role="user", parts=[Part(text=question)])
    
    response = ""
    thinking = ""
    
    async for event in runner.run_async(
        user_id="demo_user",
        session_id=session.id,
        new_message=user_content
    ):
        if hasattr(event, 'content') and event.content:
            if hasattr(event.content, 'parts'):
                for part in event.content.parts:
                    # 获取思考过程
                    if hasattr(part, 'thought') and part.thought:
                        thinking += part.text if hasattr(part, 'text') else ""
                    # 获取最终回答
                    elif hasattr(part, 'text') and part.text:
                        response += part.text
    
    # 显示思考过程
    if show_thinking and thinking:
        print(f"💭 思考过程:\n{thinking}\n")
        print("-" * 40)
    
    return response

print("✅ 辅助函数已定义（支持显示思考过程）")

✅ 辅助函数已定义（支持显示思考过程）


## 7. 测试：LOW 级别（日常使用推荐）

In [7]:
agent_low = create_agent("LOW")

response = await ask(agent_low, "北京现在几点？")
print(f"🤖 [LOW] 回答: {response}")

🤖 [LOW] 回答: 现在北京时间是 22:30。


## 8. 测试：HIGH 级别（复杂推理）

In [8]:
agent_high = create_agent("HIGH")

response = await ask(agent_high, "如果北京是晚上10点，纽约是什么时候？解释时差原因。")
print(f"🤖 [HIGH] 回答: {response}")

🤖 [HIGH] 回答: 北京晚上10点（22:00），纽约是当日**上午10点**（目前处于夏令时）。如果是冬令时，则为上午9点。

**时差原因：**
1. **地球自转**：地球自西向东自转，不同经度的地区看到日出的时间不同。
2. **时区划分**：为统一时间，全球分为24个时区。北京位于**东八区 (UTC+8)**，纽约位于**西五区 (UTC-5)**。
3. **夏令时**：纽约在夏季会进入夏令时，将时钟拨快一小时（变为UTC-4），因此两地目前的时差为12小时。


## 9. 测试：数学计算

In [9]:
response = await ask(agent_low, "计算 (15 * 8) + (42 / 7)")
print(f"🤖 [LOW] 回答: {response}")

🤖 [LOW] 回答: 结果是 126。


## 10. 查看思考过程对比

HIGH 级别会有更详细的思考过程：

In [10]:
print("🔵 LOW 级别思考:")
agent_low = create_agent("LOW")
response = await ask(agent_low, "1000 除以 7 等于多少？保留两位小数。", show_thinking=True)
print(f"🤖 回答: {response}")

🔵 LOW 级别思考:
🤖 回答: 1000 除以 7 约等于 142.86。


In [11]:
print("🔴 HIGH 级别思考:")
agent_high = create_agent("HIGH")
response = await ask(agent_high, "1000 除以 7 等于多少？保留两位小数。", show_thinking=True)
print(f"🤖 回答: {response}")

🔴 HIGH 级别思考:
🤖 回答: 1000 除以 7 等于 142.86。


## 11. 总结

### 选择建议

```
简单聊天  →  MINIMAL（最省钱最快）
日常任务  →  LOW（推荐默认）
分析任务  →  MEDIUM
复杂推理  →  HIGH（最准但最慢）
```

### 思考过程显示

```python
# part.thought = True 表示这是思考内容
for part in event.content.parts:
    if hasattr(part, 'thought') and part.thought:
        print("思考:", part.text)
```

### 与 Gemini 2.5 的区别

| 版本 | 参数 | 方式 |
|------|------|------|
| Gemini 3 | `thinking_level` | 级别选择 |
| Gemini 2.5 | `thinking_budget` | token 预算 |

**注意：参数不能混用！**

## 参考资料

- [Gemini 3 Flash 官方公告](https://blog.google/products/gemini/gemini-3-flash/)
- [ADK 模型配置文档](https://google.github.io/adk-docs/agents/models/)
- [TechCrunch 报道](https://techcrunch.com/2025/12/17/google-launches-gemini-3-flash-makes-it-the-default-model-in-the-gemini-app/)